In [3]:
import os
import glob
import re
import pandas as pd

# Ana klasör
base_dir = 'ingredient_evaluation'

all_dfs = []

for model in os.listdir(base_dir):
    model_dir = os.path.join(base_dir, model)
    if not os.path.isdir(model_dir):
        continue

    # Hem eski hem yeni sonuç dosyalarını kapsayan desen
    pattern = os.path.join(model_dir, '*_results.csv')

    for filepath in glob.glob(pattern):
        df = pd.read_csv(filepath)

        # Model bilgisini ekle
        df['model'] = model

        # 1) combo_size sütunu yoksa ground_truth_size varsa onu kopyala
        if 'combo_size' not in df.columns:
            if 'ground_truth_size' in df.columns:
                df = df.rename(columns={'ground_truth_size': 'combo_size'})
            else:
                # 2) Hâlâ yoksa dosya adından çek
                fname = os.path.basename(filepath)
                m = re.search(r'size[_\-]?(\d+)', fname)  # size_3 veya size-3
                if m:
                    df['combo_size'] = int(m.group(1))
                else:
                    # Boyut bulunamazsa NaN olarak geç; gruplanırken atlanır
                    df['combo_size'] = pd.NA

        all_dfs.append(df)

# Hepsini birleştir
big_df = pd.concat(all_dfs, ignore_index=True)

# Grupla ve ortalamaları al
summary = (
    big_df
      .dropna(subset=['combo_size'])    # boyutu bulunamayan satırları at
      .groupby(['model', 'combo_size'])
      [['hits@1', 'hits@5', 'hits@10', 'mrr']]
      .mean()
      .reset_index()
      .sort_values(['model', 'combo_size'])
)

print(summary)


                         model  combo_size    hits@1    hits@5   hits@10  \
0                        QuatE           1  0.885922  1.000000  1.000000   
1                        QuatE           2  0.638852  0.826077  0.870602   
2                        QuatE           3  0.515076  0.753931  0.831739   
3                        QuatE           4  0.498952  0.760400  0.843097   
4                       RotatE           1  0.992718  1.000000  1.000000   
5                       RotatE           2  0.914908  0.994283  0.998351   
6                       RotatE           3  0.794284  0.960794  0.983581   
7                       RotatE           4  0.724671  0.933888  0.968842   
8                       TransE           1  0.099515  0.478155  0.631068   
9                       TransE           2  0.092568  0.250660  0.345427   
10                      TransE           3  0.076148  0.217698  0.302961   
11                      TransE           4  0.087373  0.236769  0.328483   
12  text_emb

In [2]:
import os
import glob
import re
import pandas as pd

# Kök dizin
base_dir = 'WithoutIngredient'

records = []

for model in os.listdir(base_dir):
    model_dir = os.path.join(base_dir, model)
    if not os.path.isdir(model_dir):
        continue

    # Hem eski hem yeni isimlendirmeyi yakalayan desen
    pattern = os.path.join(model_dir, '*ground_truth_*_result*.csv')

    for path in glob.glob(pattern):
        fname = os.path.basename(path)

        # Regex ile ground_truth_<N> kısmındaki N’i al
        m = re.search(r'ground_truth_(\d+)', fname)
        if not m:
            continue                        # Beklenmeyen format → atla
        size = int(m.group(1))

        # CSV’yi oku ve metrik ortalamalarını al
        df  = pd.read_csv(path)
        avg = df[['hits@1', 'hits@5', 'hits@10', 'mrr']].mean()

        records.append({
            'model'          : model,
            'ground_truth_size': size,
            'avg_hits@1'     : avg['hits@1'],
            'avg_hits@5'     : avg['hits@5'],
            'avg_hits@10'    : avg['hits@10'],
            'avg_mrr'        : avg['mrr'],
        })

# Sonuç DataFrame’i
summary_df = (
    pd.DataFrame(records)
      .sort_values(['model', 'ground_truth_size'])
      .reset_index(drop=True)
)

print(summary_df)


         model  ground_truth_size  avg_hits@1  avg_hits@5  avg_hits@10  \
0        QuatE                  1    0.977273    1.000000     1.000000   
1        QuatE                  2    0.969735    0.988651     0.988651   
2        QuatE                  3    0.910379    0.950422     0.959412   
3        QuatE                  4    0.755459    0.875739     0.909458   
4        QuatE                  5    0.576605    0.777308     0.838296   
5       RotatE                  1    1.000000    1.000000     1.000000   
6       RotatE                  2    0.976040    0.991173     0.991173   
7       RotatE                  3    0.892400    0.958867     0.969218   
8       RotatE                  4    0.721519    0.885413     0.917583   
9       RotatE                  5    0.529453    0.777630     0.839261   
10      TransE                  1    0.863636    0.954545     0.977273   
11      TransE                  2    0.829760    0.935687     0.952081   
12      TransE                  3    0